In [ ]:
import pandas as pd

In [ ]:
# upload the file into colab first mate!
# https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2 dataset site
# https://data.cityofchicago.org/Public-Safety/Crimes-2023/xguy-4ndq/about_data this is the specific dataset site (only for 2023)
df1 = pd.read_csv('Chicago crimes 2023.csv')

In [ ]:
df1.columns
df1.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13203321,JG415333,09/06/2023 05:00:00 PM,002XX N Wells st,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,42.0,32,14,1174694.0,1901831.0,2023,11/04/2023 03:40:18 PM,41.886018,-87.633938,"(41.886018055, -87.633937881)"
1,13210088,JG423627,08/31/2023 12:00:00 PM,023XX W JACKSON BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,STREET,False,False,...,27.0,28,11,1160870.0,1898642.0,2023,09/16/2023 03:41:56 PM,41.877565,-87.684791,"(41.877565108, -87.68479102)"
2,13210004,JG422532,07/24/2023 09:45:00 PM,073XX S JEFFERY BLVD,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,False,False,...,7.0,43,02,1190812.0,1856743.0,2023,09/16/2023 03:41:56 PM,41.761919,-87.576209,"(41.7619185, -87.576209245)"
3,13210062,JG423596,08/27/2023 07:00:00 AM,034XX N LAWNDALE AVE,0820,THEFT,$500 AND UNDER,APARTMENT,False,False,...,30.0,21,06,1151117.0,1922554.0,2023,09/16/2023 03:41:56 PM,41.943379,-87.719974,"(41.943378528, -87.7199738)"
4,13210107,JG411849,09/04/2023 09:30:00 PM,053XX S HOMAN AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE - GARAGE,False,False,...,14.0,63,14,1154617.0,1869046.0,2023,09/16/2023 03:41:56 PM,41.796477,-87.708541,"(41.796477414, -87.708540915)"


In [ ]:
# Checking for null values
print(f'Shape of the df is: {df1.shape}')
df1.isnull().sum()

Shape of the df is (216509, 22)


ID                        0
Case Number               0
Date                      0
Block                     0
IUCR                      0
Primary Type              0
Description               0
Location Description    956
Arrest                    0
Domestic                  0
Beat                      0
District                  0
Ward                      3
Community Area            0
FBI Code                  0
X Coordinate            162
Y Coordinate            162
Year                      0
Updated On                0
Latitude                162
Longitude               162
Location                162
dtype: int64

Getting rid of columns that i think we do not need

Creating a pipeline i.e. df1->df2-> so on, so that we always have the option to go back a step.

Every new change to a df is stored in a new df (we use df.copy() in some cases, or else we just use assignment.)

In [ ]:
df2 = df1.drop(['Case Number','IUCR','Description',
                'Beat','Ward','FBI Code',
                'Updated On', 'District','Location'],
                axis = 1)

In [ ]:
df2.columns

Index(['ID', 'Date', 'Block', 'Primary Type', 'Location Description', 'Arrest',
       'Domestic', 'Community Area', 'X Coordinate', 'Y Coordinate', 'Year',
       'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
print(f'Shape of the df is {df2.shape}')

Shape of the df is (216509, 13)


In [ ]:
# rename column 'Primary type' to 'Crime type'
# rename column 'Location Description' to 'Location type' (residence, apartment, garrage, etc)

Renaming columns to make it easier for our understanding and also remove spaces in column names since it is not a good practice.

In [ ]:
df3 = df2.rename(columns={'Primary Type':'Crime_type', 'Location Description':'Location_type',
                    'Community Area':'Community_area'})

In [ ]:
df3.columns

Index(['ID', 'Date', 'Block', 'Crime_type', 'Location_type', 'Arrest',
       'Domestic', 'Community_area', 'X Coordinate', 'Y Coordinate', 'Year',
       'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
# need to split date and add new column time and month if you can (or tableau can parse the date for you)

Converting the 'Date' column into pandas Date datatype.

In [ ]:
df3.Date.unique()

array(['09/06/2023 05:00:00 PM', '08/31/2023 12:00:00 PM',
       '07/24/2023 09:45:00 PM', ..., '07/14/2023 11:05:00 AM',
       '07/18/2023 02:52:00 AM', '07/27/2023 04:02:00 AM'], dtype=object)

So we have date in the format '09/06/2023 05:00:00 PM'
we want to get rid of the merediem i.e. AM/PM.(in order to use the pd.datetime() function)

so we write a function to convert our time into a 24hr clock. And apply it on our 'Date' column

We do this because we have a function in pandas called:
to_datetime() this is how it works (we are giving it our customized format):  

pd.to_datetime('09/06/2023 5:00:00',
              format='%m/%d/%Y %H:%M:%S')


In [ ]:
def Time_converter_to_24hr_clock(x):
  date_time_merediem = x.split(' ')

  date = date_time_merediem[0]
  time = date_time_merediem[1]
  merediem = date_time_merediem[2]
# now we have time and merediem to work with

  split_time = time.split(':')
  hours = int(split_time[0])
  minutes = split_time[1]
  seconds = split_time[2]
# now we have hours and merediem to work with

  if(merediem == 'AM'): # we see if we have AM
    if(hours == 12):# we change only if its 12 AM and return 00
      result = '00'+':'+minutes+':'+seconds
    else:# if it is AM and not 12, we just return the same hour
      result = str(hours)+':'+minutes+':'+seconds
  elif (merediem == 'PM'): # we see if we have PM
    if(hours == 12): # if it is 12 and PM we need to return 12 and not 24
      result = '12'+':'+minutes+':'+seconds
    else:# if it is PM and not 12, we add 12 to hours and return
      result = str(hours+12)+':'+minutes+':'+seconds

  return date+' '+result # result is gonna be 'date new_hours:old_min:old_sec'

In [ ]:
df4 = df3.copy()
df4['Date'] = df3.Date.apply(Time_converter_to_24hr_clock)
df4['Date'] = pd.to_datetime(df4['Date'], format = '%m/%d/%Y %H:%M:%S')

In [ ]:
# need to split block, try to exract only street name

'Block' column has '002XX N Wells st', so we can extract the street by removing the first token
.

In [ ]:
def Block_to_street_converter(x):
  Block_split = x.split(' ')
  Street_name = ''
  for i in range(len(Block_split)-1):
    Street_name = Street_name + Block_split[i+1] + ' '
  return Street_name

In [ ]:
df5 = df4.copy()
df5['Block'] = df4['Block'].apply(Block_to_street_converter)
df5.rename(columns={'Block':'Street'}, inplace = True)

In [ ]:
#lets take a look at the dtypes
df5.dtypes

ID                         int64
Date              datetime64[ns]
Street                    object
Crime_type                object
Location_type             object
Arrest                      bool
Domestic                    bool
Community_area             int64
X Coordinate             float64
Y Coordinate             float64
Year                       int64
Latitude                 float64
Longitude                float64
dtype: object

In [ ]:
# there were inconsistencies with the data in "Street" column, Same street name was
# represented both in upper case letters and lower case letters.
# Converting all values in the "Street" Column into lowecase letters to deal wiht this inconsistency.

df5.Street = df5.Street.str.lower()
matching_rows = df5[df5['Street'].str.contains('s michigan ave', case=False, na=False)]
df5.shape

(216509, 13)

In [ ]:
df5['Crime_type'].value_counts()

THEFT                                46875
BATTERY                              36950
CRIMINAL DAMAGE                      25057
MOTOR VEHICLE THEFT                  24597
ASSAULT                              18906
DECEPTIVE PRACTICE                   13330
OTHER OFFENSE                        12788
ROBBERY                               9031
WEAPONS VIOLATION                     7389
BURGLARY                              6108
NARCOTICS                             4165
CRIMINAL TRESPASS                     3899
OFFENSE INVOLVING CHILDREN            1461
CRIMINAL SEXUAL ASSAULT               1328
SEX OFFENSE                           1126
PUBLIC PEACE VIOLATION                 737
HOMICIDE                               526
INTERFERENCE WITH PUBLIC OFFICER       484
STALKING                               425
ARSON                                  418
INTIMIDATION                           209
PROSTITUTION                           197
CONCEALED CARRY LICENSE VIOLATION      170
LIQUOR LAW 

In [ ]:
plt.hist(df5['Crime_type'].value_counts())

In [ ]:
df5.to_csv('Chicago crimes 2023 cleaned with cordinates.csv', header = True, index = True)